In [ ]:
import pandas as pd 
import pathlib
import win32com.client as win32 

# IMPORTANTO BASE DE DADOS 
emails = pd.read_excel(r'c:\.xlsx') # SELECIONA ONDE ESTA O ARQUIVO 
lojas = pd.read_csv(r'C:\.csv', sep=';', encoding= 'latin1')
vendas = pd.read_excel(r'C:\.xlsx')

# TRATANDO BASE DE DADOS 

# incluindo nome da loja em vendas
vendas = vendas.merge(lojas, on='ID Loja')
# criar tabela pra cada loja 
dicionario_lojas = {}

for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]

# definindo dia do indicador
dia_indicador = vendas['Data'].max() 

# FAZENDO BACKUP NO COMPUTADOR
# identificar se a pasta ja existe

caminho_backup = pathlib.Path(r'C:\Users\Win\Downloads\Curso da Hashtag\Projetos\Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja 
        nova_pasta.mkdir()
        
    # salvar dentro da pasta
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    
    dicionario_lojas[loja].to_excel(local_arquivo)

# METAS DA EMPRESA 

meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdprodutos_dia = 4
meta_qtdprodutos_ano = 120
meta_ticketmedio_ano = 500
meta_ticketmedio_dia = 500


# CALCULANDO INDICADOR

loja = ''
vendas_loja = dicionario_lojas[loja]
vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador,:]

# Faturamento 
faturamento_ano = vendas_loja['Valor Final'].sum()

faturamento_dia = vendas_loja_dia['Valor Final'].sum()

# Diversidade de produtos 
qtde_produtos_ano = len(vendas_loja['Produto'].unique()) # tira os valores duplicados 

qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())


# Ticket medio
valor_venda = vendas_loja.groupby('Código Venda').sum()
ticket_medio_ano = valor_venda['Valor Final'].mean()
display(ticket_medio_ano)

# ticket_medio_dia  
valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
display(ticket_medio_dia)


# ENVIANDO O EMAIL PARA CADA LOJA SEPARADO

for loja in dicionario_lojas:
    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador,:]

    # Faturamento 
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    # Diversidade de produtos 
    qtde_produtos_ano = len(vendas_loja['Produto'].unique()) # tira os valores duplicados 
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())


    # Ticket medio
    valor_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    

    # ticket_medio_dia  
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
   
    outlook = win32.Dispatch('outlook.application')

    nome = emails.loc[emails['Loja']==loja,'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = emails.loc[emails['Loja']==loja,'E-mail'].values[0]
    mail.Subject = f'OnePage dia{dia_indicador.day}/{dia_indicador.month}, Loja{loja}'

    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if qtde_produtos_dia >= meta_qtdprodutos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    if qtde_produtos_ano >= meta_qtdprodutos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'
    if ticket_medio_dia >= meta_ticketmedio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'
    if ticket_medio_ano >= meta_ticketmedio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'

    mail.HTMLBody = f'''
    <p> Bom dia, {nome}</p>

    <p> O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi: </p> 

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenario Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color = "{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td >Diversidade Produtos</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{meta_qtdprodutos_dia}</td>
        <td style="text-align: center"><font color = "{cor_qtde_dia}">◙</font></td>
      </tr>
        <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
        <td style="text-align: center"><font color = "{cor_ticket_dia}">◙</font></td>
      </tr>
    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenario Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color = "{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td >Diversidade Produtos</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{meta_qtdprodutos_ano}</td>
        <td style="text-align: center"><font color = "{cor_qtde_ano}">◙</font></td>
      </tr>
        <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
        <td style="text-align: center"><font color = "{cor_ticket_ano}">◙</font></td>
      </tr>
    </table>

    <p> Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p> Qualquer duvida estou a disposicao.</p>
    <p.> Att., Nelson</p>
    '''

    attachment = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))
    mail.Send()
    print(f'Email da loja {loja} enviado')
    
    
# CRIANDO RANKING PARA A DIRETORIA 

faturamento_loja_ano = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_loja_ano = faturamento_loja_ano.sort_values(by='Valor Final', ascending=False) # do maior para o menor
display(faturamento_loja_ano)

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day)   
faturamento_loja_ano.to_excel(r'C:\Users\Win\Downloads\Curso da Hashtag\Projetos\Backup Arquivos Lojas\{}'.format(nome_arquivo))

vendas_dia = vendas.loc[vendas['Data']== dia_indicador, :]
faturamento_loja_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_loja_dia = faturamento_loja_dia.sort_values(by='Valor Final', ascending=False)
display(faturamento_loja_dia)

nome_arquivo = '{}_{}_Ranking Diario.xlsx'.format(dia_indicador.month, dia_indicador.day)   
faturamento_loja_dia.to_excel(r'C:\Users\Win\Downloads\Curso da Hashtag\Projetos\Backup Arquivos Lojas\{}'.format(nome_arquivo))

# ENVIANDO EMAIL PARA A DIRETORIA 

outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']=='Diretoria','E-mail'].values[0]
mail.Subject = f'Ranking dia {dia_indicador.day}/{dia_indicador.month}'
mail.Body = f'''
Prezados, bom dia.


Melhor loja do Dia em Faturamento: Loja {faturamento_loja_dia.index[0]} com Faturamento R${faturamento_loja_dia.iloc[0,0]:.2f}
Pior loja do Dia em Faturamento: Loja {faturamento_loja_dia.index[-1]} com Faturamento R${faturamento_loja_dia.iloc[-1,0]:.2f}

Melhor loja do ano em Faturamento: Loja {faturamento_loja_ano.index[0]} com Faturamento R${faturamento_loja_ano.iloc[0,0]:.2f}
Pior loja do Ano em Faturamento: Loja {faturamento_loja_ano.index[-1]} com Faturamento R${faturamento_loja_ano.iloc[-1,0]:.2f}


Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou a disposiçao

att.,
Nelson.''' 

attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Diario.xlsx'
mail.Attachments.Add(str(attachment))
mail.Send()
print(f'Email da Diretoria enviado')